### Парсинг телефонов с Авито

In [1]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

import time

import os
import re

from tqdm import tqdm

# Библиотеки для парсинга 
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager

# Парсер 
from tools import get_ad_urls, get_photos
from parser import parse_avito_page

### Константы

In [2]:
URL_TEMPLATE = "https://www.avito.ru/sankt-peterburg/telefony/mobilnye_telefony/apple-ASgBAgICAkS0wA3OqzmwwQ2I_Dc?p="
PAUSE_DURATION_SECONDS = 2
NUM_PAGES = 100
product_columns = [
'title', 'price', 'characteristics', 
'description', 'views', 'date', 'rating',
'location', 'link', 'seller_id', 'today_views'
, 'about'
]

seller_columns = [
    'seller_id', 'name', 'rating', 'reviews',
    'subscribers', 'subscriptions', 'registered', 
    'done_deals', 'active_deals', 'docs_confirmed',
    'phone_confirmed', 'response_time'
]


### Запуск сервиса

In [3]:
service = Service(GeckoDriverManager().install())

# dont open browser
options = webdriver.FirefoxOptions()
# options.add_argument('--headless')
driver = webdriver.Firefox(service=service, options=options)

### Максимальный номер сохраненной страницы  

In [4]:
output_folder = "data8"
os.makedirs(output_folder, exist_ok=True)

# Получаем список всех файлов в папке
files = os.listdir(output_folder)

# Ищем файлы с расширением .csv и извлекаем числовые части
numbers = []
for file in files:
    if file.endswith('.parquet'):
        match = re.search(r'\d+', file)
        if match:
            numbers.append(int(match.group()))

# Находим максимальное число
max_number = max(numbers) if numbers else None
print(f"Максимальный номер: {max_number}")


if not max_number: 
    start_page_number = 1
else: 
    start_page_number = max_number + 1

end_page_number = start_page_number + NUM_PAGES

print(f"Начнем парсинг со страницы номер: {start_page_number} (вкл)")
print(f"Закончим парсинг на странице номер: {end_page_number} (не вкл)")

Максимальный номер: 20
Начнем парсинг со страницы номер: 21 (вкл)
Закончим парсинг на странице номер: 121 (не вкл)


### Парсинг

In [5]:
print(output_folder)

for page_num in range(20, end_page_number):
    
    print(f"Начали парсинг страницы # {page_num}")

    df_page = pd.DataFrame(columns=product_columns)
    df_seller = pd.DataFrame(columns=seller_columns)

    try:
        # Загрузка страницы с объявлениями
        url = URL_TEMPLATE + str(page_num)
        driver.get(url)
        time.sleep(PAUSE_DURATION_SECONDS)

        # Получаем HTML-код страницы
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
  
        # Извлекаем ссылки на объявления на текущей странице
        links = get_ad_urls(soup)

        for link in tqdm(links):
            
            try:
                # Переход на страницу объявления
                driver.get(link)
                time.sleep(PAUSE_DURATION_SECONDS)  # Задержка для полной загрузки

                # Парсим данные на странице объявления (название, цена, фото, описание и т.д.)
                ad_data, seller_data = parse_avito_page(driver=driver) # <- словарик 
                ad_data['link'] = link
                # ad_data['photo'] = get_photos(driver=driver, cnt=5)
                
                df_page = pd.concat([df_page, pd.DataFrame([ad_data])], ignore_index=True)
                df_seller = pd.concat([df_seller, pd.DataFrame([seller_data])], ignore_index=True)
            except TimeoutException:
                print(f"Ошибка: объявление {link} не загрузилось, пропускаем...")
                continue
            except Exception as e:
                print(f"Произошла ошибка при обработке объявления {link}: {e}")
                continue

        # Сохраняем данные для текущей страницы в DataFrame
        df_page.to_parquet(f'{output_folder}/phones_data_page_{page_num}.parquet')
        df_seller.to_parquet(f'{output_folder}/sellers_data_page_{page_num}.parquet')
    except TimeoutException:
        print(f"Ошибка: страница {page_num} не загрузилась, пропускаем...")
        continue
    except Exception as e:
        print(f"Произошла ошибка на странице {page_num}: {e}")
        continue



data8
Начали парсинг страницы # 20


0it [00:00, ?it/s]


Начали парсинг страницы # 21


0it [00:00, ?it/s]


Начали парсинг страницы # 22


...

Начали парсинг страницы # 23


  6%|▌         | 3/50 [02:04<30:51, 39.40s/it]

Произошла ошибка при обработке объявления https://www.avito.ru/sankt-peterburg/telefony/iphone_11_pro_max_64_gb_4428627755: HTTPConnectionPool(host='localhost', port=56169): Read timed out. (read timeout=120)
Произошла ошибка при обработке объявления https://www.avito.ru/sankt-peterburg/telefony/iphone_11_128_gb_4432116540: Message: Browsing context has been discarded
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchWindowError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:679:5
assert.that/<@chrome://remote/content/shared/webdriver/Assert.sys.mjs:515:13
assert.open@chrome://remote/content/shared/webdriver/Assert.sys.mjs:147:4
GeckoDriver.prototype.navigateTo@chrome://remote/content/marionette/driver.sys.mjs:863:39
despatch@chrome://remote/content/marionette/server.sys.mjs:318:40
execute@chrome://remote/content/marionette/server.sys.mjs:289:16
onPacket/<@chrome://r

100%|██████████| 50/50 [02:04<00:00,  2.49s/it]


Произошла ошибка при обработке объявления https://www.avito.ru/sankt-peterburg/telefony/iphone_13_pro_max_128_gb_4596258322: Message: Browsing context has been discarded
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchWindowError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:679:5
assert.that/<@chrome://remote/content/shared/webdriver/Assert.sys.mjs:515:13
assert.open@chrome://remote/content/shared/webdriver/Assert.sys.mjs:147:4
GeckoDriver.prototype.navigateTo@chrome://remote/content/marionette/driver.sys.mjs:863:39
despatch@chrome://remote/content/marionette/server.sys.mjs:318:40
execute@chrome://remote/content/marionette/server.sys.mjs:289:16
onPacket/<@chrome://remote/content/marionette/server.sys.mjs:262:20
onPacket@chrome://remote/content/marionette/server.sys.mjs:263:9
_onJSONObjectReady/<@chrome://remote/content/marionette/transport.sys.mjs:494:20

Произош

In [7]:
driver.quit()

In [11]:
# read data from parquet files
data = []
for file in os.listdir(output_folder):
    if file.endswith('.parquet'):
        df = pd.read_parquet(f'{output_folder}/{file}')
        data.append(df)

# get parquet files in 
data[0].head(20)


,seller_id,name,rating,reviews,subscribers,subscriptions,registered,done_deals,active_deals,docs_confirmed,phone_confirmed,response_time
0,fccdb8de-ef33-47ea-8073-a78434d1c9b0,Unknown,"3,8",None,None,None,None,None,None,False,False,Отвечает за несколько часов
